In [1]:
#Imports for the project
import pandas as pd
import matplotlib as mpl
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load the CSV into a DataFrame
df = pd.read_csv(r'games.csv')

In [3]:
df.shape

(61326, 18)

In [4]:
#A copy of the data for backup
new_df = df.iloc[:,1:].copy()

In [5]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61326 entries, 0 to 61325
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Game            61326 non-null  object 
 1   Pos             61326 non-null  int64  
 2   Genre           61326 non-null  object 
 3   Console         61326 non-null  object 
 4   Publisher       61326 non-null  object 
 5   Developer       61300 non-null  object 
 6   VGChartz Score  1216 non-null   float64
 7   Critic Score    6554 non-null   float64
 8   User Score      391 non-null    float64
 9   Total Shipped   3842 non-null   object 
 10  Total Sales     18918 non-null  object 
 11  NA Sales        12639 non-null  object 
 12  PAL Sales       12824 non-null  object 
 13  Japan Sales     6794 non-null   object 
 14  Other_Sales     15163 non-null  object 
 15  Release_Date    56668 non-null  object 
 16  Last_Update     15078 non-null  object 
dtypes: float64(3), int64(1), object

In [6]:
#When the value Null appears in both 'Total Shipped' & 'total Sales' fields - we delete these rows
# **we must provide data in at least one of the fields as part of our prediction question
new_df.dropna(subset=['Total Shipped','Total Sales'], inplace = True ,how='all')

#After checking thr data we saw a small amount of Null's values,
#so we decided to delete rows when both of the fields consist Null value - it doesn't harm in our data processing
new_df.dropna(subset=['Release_Date','Last_Update'], inplace = True ,how='all')
new_df.dropna(subset=['Developer'], inplace = True)

#Critical fields for our predict question and therefore when there is no data - irrelevant
new_df.dropna(subset=['Game'], inplace = True)
new_df.dropna(subset=['Console'], inplace = True)

In [7]:
#Replacing the fields values with an empty value to unify them and then deleting the irrelevant fields for prediction efficiency
new_df['Total Shipped'] = new_df['Total Shipped'].fillna("")
new_df['Total Sales'] = new_df['Total Sales'].fillna("")


new_df['Total_Sales']= new_df['Total Shipped'] + new_df['Total Sales']

#Delete the 2 fields after merging
del new_df['Total Sales']
del new_df['Total Shipped']


In [8]:
#Calaulates the amount of duplicates in 'Game'&'Console
new_df.duplicated(['Game','Console']).sum()

5160

In [9]:
#Summing & sorting the amount of Null's, then deleted the rows with the highest amount of Null's values and drop duplicates
new_df['sum_of_null'] = new_df.apply(lambda x: x.isna().sum(), axis=1)
new_df = new_df.sort_values(by=['sum_of_null']).drop_duplicates(['Game','Console'], keep='first').sort_index().reset_index(drop=True)
del new_df['sum_of_null']

In [10]:
#Checking there are no duplicates
new_df.duplicated(['Game','Console']).sum()

0

In [11]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17510 entries, 0 to 17509
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Game            17510 non-null  object 
 1   Pos             17510 non-null  int64  
 2   Genre           17510 non-null  object 
 3   Console         17510 non-null  object 
 4   Publisher       17510 non-null  object 
 5   Developer       17510 non-null  object 
 6   VGChartz Score  450 non-null    float64
 7   Critic Score    3705 non-null   float64
 8   User Score      231 non-null    float64
 9   NA Sales        9227 non-null   object 
 10  PAL Sales       9349 non-null   object 
 11  Japan Sales     4808 non-null   object 
 12  Other_Sales     11001 non-null  object 
 13  Release_Date    17057 non-null  object 
 14  Last_Update     5498 non-null   object 
 15  Total_Sales     17510 non-null  object 
dtypes: float64(3), int64(1), object(12)
memory usage: 2.1+ MB


In [12]:
#Due to the multiplicity of missing values, we have reset all the Null values
new_df['VGChartz Score'] = new_df['VGChartz Score'].fillna(0)
new_df['Critic Score'] = new_df['Critic Score'].fillna(0)
new_df['User Score'] = new_df['User Score'].fillna(0)
new_df['NA Sales'] = new_df['NA Sales'].fillna(0)
new_df['PAL Sales'] = new_df['PAL Sales'].fillna(0)
new_df['Japan Sales'] = new_df['Japan Sales'].fillna(0)
new_df['Other_Sales'] = new_df['Other_Sales'].fillna(0)
new_df['Release_Date'] = new_df['Release_Date'].fillna(new_df['Last_Update'])
del new_df['Last_Update']


In [13]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17510 entries, 0 to 17509
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Game            17510 non-null  object 
 1   Pos             17510 non-null  int64  
 2   Genre           17510 non-null  object 
 3   Console         17510 non-null  object 
 4   Publisher       17510 non-null  object 
 5   Developer       17510 non-null  object 
 6   VGChartz Score  17510 non-null  float64
 7   Critic Score    17510 non-null  float64
 8   User Score      17510 non-null  float64
 9   NA Sales        17510 non-null  object 
 10  PAL Sales       17510 non-null  object 
 11  Japan Sales     17510 non-null  object 
 12  Other_Sales     17510 non-null  object 
 13  Release_Date    17510 non-null  object 
 14  Total_Sales     17510 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 2.0+ MB


In [14]:
new_df.shape

(17510, 15)

In [15]:
#change total sales to integer  for example "51.00m" -> 51,000,
def func(x):
    if (type(x) != str):
        return 0
    result = int(''.join(filter(str.isdigit, x))) * 10000
    return result
new_df['NA Sales'] = new_df.apply(lambda row: func(row['NA Sales']),axis=1)
new_df['PAL Sales'] = new_df.apply(lambda row: func(row['PAL Sales']),axis=1)
new_df['Japan Sales'] = new_df.apply(lambda row: func(row['Japan Sales']),axis=1)
new_df['Other_Sales'] = new_df.apply(lambda row: func(row['Other_Sales']),axis=1)
new_df['Total_Sales'] = new_df.apply(lambda row: func(row['Total_Sales']),axis=1)
new_df = new_df.rename(columns = {'NA Sales' : 'NA_Sales','PAL Sales' : 'PAL_Sales','Japan Sales': 'Japan_Sales','User Score':'User_Score','Critic Score' :'Critic_Score' ,'VGChartz Score':'VGChartz_Score' })

In [16]:
new_df['Total_Sales']

0        51000000
1        47820000
2        25200000
3        21150000
4        20000000
           ...   
17505       30000
17506       30000
17507       40000
17508       30000
17509       10000
Name: Total_Sales, Length: 17510, dtype: int64

In [17]:
#Replacing the month values to numeric values and splitting the release_date field for the machine learning phase
test = {"Jan" :1,
        "Feb" :2,
        "Mar" :3,
        "Apr" :4,
        "May" :5,
        "Jun" :6,
        "Jul" :7,
        "Aug" :8,
        "Sep" :9,
        "Oct" :10,
        "Nov" :11,
        "Dec" :12}
def str_to_month(date):
    month = date.split(" ")[1]
    
    return test[month] 

def str_to_year(date):
    year = int(date.split(" ")[2])
    if (year > 30):
        return year + 1900
    else:
        return year + 2000
    
new_df['Relese_Month'] = new_df.apply(lambda row: str_to_month(row['Release_Date']),axis=1)
new_df['Relese_Year'] = new_df.apply(lambda row: str_to_year(row['Release_Date']),axis=1)
del new_df['Release_Date']

In [18]:
#add column of Hit games and insert '1' for every game that have total_sales greater then 1m
def hit_games(sales):
    if (sales >= 1000000):
        return 1
    else:
        return 0
new_df['Hit_Games'] = new_df.apply(lambda row: hit_games(row['Total_Sales']),axis=1)


In [19]:
new_df

,Game,Pos,Genre,Console,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Total_Sales,Relese_Month,Relese_Year,Hit_Games
0,God of War,1,Action,Series,Sony Computer Entertainment,SIE Santa Monica Studio,0.0,0.0,0.0,0,0,0,0,51000000,3,2005,1
1,Warriors,2,Action,Series,KOEI,Omega Force,0.0,0.0,0.0,0,0,0,0,47820000,6,1997,1
2,Devil May Cry,3,Action,Series,Capcom,Capcom,0.0,0.0,0.0,0,0,0,0,25200000,10,2001,1
3,Dynasty Warriors,4,Action,Series,KOEI,Omega Force,0.0,0.0,0.0,0,0,0,0,21150000,6,1997,1
4,Frogger,5,Action,Series,Konami,Konami,0.0,0.0,0.0,0,0,0,0,20000000,10,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17505,Wand of Fortune 2 FD: Kimi ni Sasageru Epilogue,461,Visual+Novel,PSP,Idea Factory,Idea Factory,0.0,0.0,0.0,0,0,30000,0,30000,11,2012,0
17506,Infinite Stratos 2: Ignition Hearts,462,Visual+Novel,PSV,5pb,5pb. Games,0.0,0.0,0.0,0,0,30000,0,30000,2,2014,0
17507,Shin Hayarigami,463,Visual+Novel,PS3,Nippon Ichi Software,Nippon Ichi Software,0.0,0.0,0.0,0,0,40000,0,40000,8,2014,0
17508,Root Letter,464,Visual+Novel,PS4,PQube,Kadokawa Games,0.0,0.0,0.0,0,0,30000,0,30000,11,2016,0


In [20]:
#check if we have outliers in sales
new_df.describe(include = "all")

,Game,Pos,Genre,Console,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Total_Sales,Relese_Month,Relese_Year,Hit_Games
count,17510,17510.000000,17510,17510,17510,17510,17510.000000,17510.000000,17510.000000,1.751000e+04,1.751000e+04,1.751000e+04,1.751000e+04,1.751000e+04,17510.000000,17510.000000,17510.000000
unique,12914,NaN,18,47,1081,3304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Minecraft,NaN,Action,PC,Ubisoft,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,11,NaN,3218,2610,833,490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2107.241748,NaN,NaN,NaN,NaN,0.194466,1.542479,0.111062,1.389406e+05,7.940206e+04,2.710109e+04,2.688921e+04,1.301541e+06,7.092119,2008.049743,0.173558
std,NaN,1972.332540,NaN,NaN,NaN,NaN,1.218351,3.050528,0.973822,3.781142e+05,2.966896e+05,9.494074e+04,1.007367e+05,9.785694e+06,3.481718,6.974279,0.378740
min,NaN,1.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1970.000000,0.000000
25%,NaN,387.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+04,4.000000,2004.000000,0.000000
50%,NaN,1562.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.700000e+05,8.000000,2009.000000,0.000000
75%,NaN,3299.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.300000e+05,4.000000e+04,1.000000e+04,2.000000e+04,5.500000e+05,10.000000,2013.000000,0.000000


In [21]:
new_df.to_csv("clean_data.csv")

In [ ]:
##################################################################